# EDA bank-marketing-campaign-data

## Set up

In [471]:
#!pip install edastatmil_milser
!pip install --upgrade edastatmil_milser
color = "#254025"
import warnings
warnings.filterwarnings('ignore') 

Defaulting to user installation because normal site-packages is not writeable


In [472]:
from edastatmil_milser import edas_tatmil as EDA

total_data_raw1 = EDA.pd.read_csv('../data/raw/tmdb_5000_credits.csv', delimiter=',')
total_data_raw2 = EDA.pd.read_csv('../data/raw/tmdb_5000_movies.csv', delimiter=',')


In [473]:
total_data_raw_merged = EDA.pd.merge(total_data_raw1, total_data_raw2, on='title')
total_data_raw = total_data_raw_merged[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

total_data_raw.to_csv(f"../data/raw/total_data_raw.csv")

In [474]:
total_data_clean = total_data_raw.copy()
EDA.explore(total_data_clean)



Rows: 4809
Columns: 7
+----------+----------------+------------+-----------+---------------+
|          | Non-Null Count | Null Count | Data Type | Data Category |
+----------+----------------+------------+-----------+---------------+
| movie_id |      4809      |     0      |   int64   |   Numerical   |
|  title   |      4809      |     0      |  object   |  Categorical  |
| overview |      4806      |     3      |  object   |  Categorical  |
|  genres  |      4809      |     0      |  object   |  Categorical  |
| keywords |      4809      |     0      |  object   |  Categorical  |
|   cast   |      4809      |     0      |  object   |  Categorical  |
|   crew   |      4809      |     0      |  object   |  Categorical  |
+----------+----------------+------------+-----------+---------------+


(['title', 'overview', 'genres', 'keywords', 'cast', 'crew'], ['movie_id'])

El dataset tiene 4809 instanias con 7 variables


In [475]:
total_data_clean = total_data_clean.dropna()
EDA.explore(total_data_clean)

Rows: 4806
Columns: 7
+----------+----------------+------------+-----------+---------------+
|          | Non-Null Count | Null Count | Data Type | Data Category |
+----------+----------------+------------+-----------+---------------+
| movie_id |      4806      |     0      |   int64   |   Numerical   |
|  title   |      4806      |     0      |  object   |  Categorical  |
| overview |      4806      |     0      |  object   |  Categorical  |
|  genres  |      4806      |     0      |  object   |  Categorical  |
| keywords |      4806      |     0      |  object   |  Categorical  |
|   cast   |      4806      |     0      |  object   |  Categorical  |
|   crew   |      4806      |     0      |  object   |  Categorical  |
+----------+----------------+------------+-----------+---------------+


(['title', 'overview', 'genres', 'keywords', 'cast', 'crew'], ['movie_id'])

In [476]:
print(total_data_raw['cast'].iloc[0])


[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 2

In [477]:
import json
#warnings.filterwarnings('default') 
warnings.filterwarnings('ignore') 

total_data_clean['genres'] = total_data_raw['genres'].apply(json.loads)
total_data_clean['genres'] = total_data_clean['genres'].apply(lambda x: [item['name'] for item in x])

total_data_clean['keywords'] = total_data_raw['keywords'].apply(json.loads)
total_data_clean['keywords'] = total_data_clean['keywords'].apply(lambda x: [item['name'] for item in x])

total_data_clean['cast'] = total_data_raw['cast'].apply(json.loads)
total_data_clean['cast'] = total_data_clean['cast'].apply(lambda x: [item['name'] for item in x[:3]])



In [478]:
warnings.filterwarnings('ignore')


#gracias internet
total_data_clean['crew'] = total_data_raw['crew'].apply(json.loads)
# Filter for rows where the job is 'Director'
directors_df = total_data_clean[total_data_clean['crew'].apply(lambda x: any(entry['job'] == 'Director' for entry in x) if isinstance(x, list) else False)]

# Replace NaN values with empty string in the 'directors_df' DataFrame
directors_df['crew'] = directors_df['crew'].apply(lambda x: x if isinstance(x, list) else '')
# Extract director names
director_names = directors_df['crew'].apply(lambda x: [entry['name'] for entry in x if entry['job'] == 'Director'][0])
####

total_data_clean.loc[directors_df.index, 'crew'] = director_names
print(type(total_data_clean['genres'][0]))
print(type(total_data_clean['cast'][0]))
print(type(total_data_clean['crew'][0]))
print(type(total_data_clean['keywords'][0]))

<class 'list'>
<class 'list'>
<class 'str'>
<class 'list'>


In [479]:

# Convertir la columna overview en una lista
total_data_clean['overview'] = total_data_raw['overview'].apply(lambda x: [x])



# Eliminar espacios entre palabras en ciertas columnas
columns_to_strip = ['genres', 'cast', 'keywords']


for col in columns_to_strip:
    total_data_clean[col] = total_data_clean[col].apply(lambda x: [''.join(word.split()) if isinstance(word, str) and ' ' in word else word for word in x])
    
# Eliminar espacios alrededor del nombre y convertirlo en una sola cadena
total_data_clean['crew'] = total_data_clean['crew'].apply(lambda x: x.replace(" ", "") if isinstance(x, str) else x)


print(type(total_data_clean['overview'][0]))
print(type(total_data_clean['genres'][0]))
print(type(total_data_clean['cast'][0]))
print(type(total_data_clean['crew'][0]))
print(type(total_data_clean['keywords'][0]))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'str'>
<class 'list'>


In [480]:
print((total_data_clean['overview'][0]))
print((total_data_clean['genres'][0]))
print((total_data_clean['cast'][0]))
print((total_data_clean['crew'][0]))
print((total_data_clean['keywords'][0]))

['In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.']
['Action', 'Adventure', 'Fantasy', 'ScienceFiction']
['SamWorthington', 'ZoeSaldana', 'SigourneyWeaver']
JamesCameron
['cultureclash', 'future', 'spacewar', 'spacecolony', 'society', 'spacetravel', 'futuristic', 'romance', 'space', 'alien', 'tribe', 'alienplanet', 'cgi', 'marine', 'soldier', 'battle', 'loveaffair', 'antiwar', 'powerrelations', 'mindandsoul', '3d']


In [481]:
# Crear la columna 'tags'
total_data_clean['tags'] = ""

# Convertir los valores de las columnas a cadenas y unir las listas
total_data_clean['overview'] = total_data_clean['overview'].apply(lambda x: ' '.join(x) if isinstance(x, list) else str(x) if not EDA.pd.isna(x) else '')
total_data_clean['genres'] = total_data_clean['genres'].apply(lambda x: ' '.join(x))
total_data_clean['cast'] = total_data_clean['cast'].apply(lambda x: ' '.join(x))
total_data_clean['crew'] = total_data_clean['crew'].apply(lambda x: str(x))  # Convertir 'crew' a cadena
total_data_clean['keywords'] = total_data_clean['keywords'].apply(lambda x: ' '.join(x))

# Crear la columna 'tags' concatenando las columnas 'overview', 'genres', 'cast', 'crew' y 'keywords'
total_data_clean['tags'] = total_data_clean['overview'] + ' ' + total_data_clean['genres'] + ' ' + total_data_clean['keywords'] + ' ' + total_data_clean['cast'] + ' ' + total_data_clean['crew'] 

# Mostrar el resultado
print(total_data_clean['tags'][0])


In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron


In [482]:
total_data_clean.to_csv('../data/processed/' + 'total_data_clean_procesed.csv', index=False)

In [485]:
# KNN modeling
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors

vectorizer = CountVectorizer()
CV_matrix = vectorizer.fit_transform(total_data_clean["tags"])

In [488]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(CV_matrix)


In [498]:
def recommend(movie):
    try:    
    
        movie_index = total_data_clean[total_data_clean["title"] == movie].index[0]
        distances = similarity[movie_index]
        movie_list = sorted(list(enumerate(distances)), reverse = True , key = lambda x: x[1])[1:6]
    
        for i in movie_list:
            print(total_data_clean.iloc[i[0]].title)
    except Exception as e:
        print("Your movie is not in the List")

In [499]:
recommend("a")

Your movie is not in the List
